In [2]:
# OM NAMAH SHIVAY!
import anthropic
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import os
import glob
from tqdm import tqdm
from scipy.stats import zscore
from meteostat import Point, Hourly
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error

In [3]:
IDX2DAY = {0:"Mon", 1:"Tue", 2:"Wed", 3:"Thu", 4:"Fri", 5:"Sat", 6:"Sun"}
IDX2TIME = {0:"06:00 - 11:59", 1:"12:00 - 17:59", 2:"18:00 - 23:59", 3:"00:00 - 05:59"}

In [29]:
df_2412 = pd.read_csv("./freq_2412.csv")
df_2412["Date"] = pd.to_datetime(df_2412["Date"])
df_2437 = pd.read_csv("./freq_2437.csv")
df_2437["Date"] = pd.to_datetime(df_2437["Date"])
df_2462 = pd.read_csv("./freq_2462.csv")
df_2462["Date"] = pd.to_datetime(df_2462["Date"])
df_2484 = pd.read_csv("./freq_2484.csv")
df_2484["Date"] = pd.to_datetime(df_2484["Date"])

json_ip = {}
for df in [df_2412, df_2437, df_2462, df_2484]:
    location = 0
    # df = df[(np.abs(zscore(df['power'])) < 2)]
    df = df[df["Location"]==location].sort_values(by="Date")
    freq = df.loc[df.index[0], "frequency"]

    print("Frequency:", freq, "# of instances:", len(df))
    print("Min:", df["power"].min(), "Max:", df["power"].max(), "Avg:", df["power"].mean())
    
    json_ip[f"power_{freq}"] = df["power"].to_list()[:2400]
    # df['Date'] = df['Date'].dt.date
    df['Date'] = df['Date'].apply(lambda x: x.isoformat())
    json_ip[f"date_{freq}"] = df["Date"].to_list()[:2400]
    print(json_ip[f'power_{freq}'])
    print(json_ip[f"date_{freq}"])
    print('\n')

Frequency: 2411.978 # of instances: 2570
Min: -105.154 Max: -69.491 Avg: -97.53172918287939
[-91.251, -93.425, -94.223, -90.782, -89.717, -91.271, -94.289, -91.365, -93.547, -94.148, -91.739, -88.748, -91.868, -94.257, -92.686, -91.081, -93.755, -93.08, -91.296, -91.182, -94.115, -89.624, -94.929, -101.219, -100.516, -91.224, -91.898, -99.745, -100.721, -93.991, -89.533, -78.686, -92.725, -96.487, -100.984, -93.241, -99.676, -92.503, -92.701, -101.961, -94.822, -93.415, -101.282, -90.601, -92.14, -100.137, -90.437, -101.084, -93.887, -94.521, -100.144, -93.537, -101.646, -99.037, -94.255, -95.174, -101.136, -101.622, -92.156, -97.826, -92.642, -101.355, -96.844, -97.776, -93.889, -94.634, -98.322, -94.022, -97.0, -94.806, -99.54, -95.398, -89.401, -101.17, -93.403, -95.44, -100.875, -91.929, -94.656, -100.552, -95.097, -95.12, -98.667, -94.406, -100.8, -94.297, -95.149, -94.357, -101.479, -88.442, -99.677, -101.838, -96.893, -100.647, -95.184, -101.769, -96.196, -94.625, -95.442, -95.8

In [17]:
json_ip

{'power_2411.978': [-102.788,
  -99.225,
  -93.652,
  -100.663,
  -97.803,
  -103.316,
  -93.305,
  -103.74,
  -101.391,
  -100.897,
  -96.648,
  -100.386,
  -101.979,
  -95.801,
  -96.4,
  -97.357,
  -91.286,
  -101.879,
  -101.511,
  -101.0,
  -99.16,
  -101.675,
  -102.587,
  -101.33,
  -98.678,
  -103.779,
  -94.547,
  -99.216,
  -97.26,
  -100.801,
  -101.214,
  -102.155,
  -100.519,
  -100.612,
  -99.355,
  -103.693,
  -100.187,
  -102.812,
  -99.084,
  -97.13,
  -102.445,
  -98.76,
  -99.337,
  -101.535,
  -101.711,
  -101.797,
  -96.106,
  -102.395,
  -101.781,
  -101.714,
  -102.399,
  -89.751,
  -99.465,
  -103.205,
  -102.317,
  -99.354,
  -103.694,
  -100.177,
  -100.466,
  -100.733,
  -103.324,
  -102.579,
  -102.57,
  -101.32,
  -103.429,
  -102.044,
  -102.13,
  -98.99,
  -100.929,
  -100.386,
  -98.988,
  -98.253,
  -98.007,
  -97.079,
  -101.876,
  -101.893,
  -102.736,
  -100.601,
  -93.319,
  -100.406,
  -102.912,
  -101.436,
  -91.962,
  -99.832,
  -101.602,
  -98.5

In [4]:
def get_avg(avg1, avg2):
    return ((avg1-avg2)/abs(avg2))*100

## Location-wise time Analysis

In [50]:
main_df = pd.read_csv("freq_2412.csv")
main_df["Date"] = pd.to_datetime(main_df["Date"])
json_ip = {}
for loc in range(1):
    df = main_df[main_df["Location"]==loc][300:2000].sort_values(by="Date")
    df = df[(np.abs(zscore(df['power'])) < 3)]
    df['Date'] = pd.to_datetime(df['Date'])
    df['Date'] = df['Date'].dt.date
    df['Date'] = df['Date'].apply(lambda x: x.isoformat())
    print(df['Date'].to_list())

    # weekday_df = df[df["Day"].isin([0,1,2,3])]
    # weekend_df = df[df["Day"].isin([4,5])]
    # df['Day'] = df['Day'].map(IDX2DAY)
    
    # df["Time"] = pd.to_datetime(df["Time"], format="%H:%M:%S").dt.time
    # # print(df["Time"])
    # df["Hour"] = df["Time"].apply(lambda t: t.hour)
    # # morn_df = df[df["Time_of_Day"]==0]
    # # noon_df = df[df["Time_of_Day"]==1]
    # # eve_df = df[df["Time_of_Day"]==2]
    # night_df = df[df["Time_of_Day"]==3]
    # morn_df = df[(df["Hour"] >= 7) & (df["Hour"] <= 18)]
    # night_df = df[((df["Hour"] >= 0) & (df["Hour"] <= 6)) | ((df["Hour"] >= 19) & (df["Hour"] <= 23))]
    # # df["Time_of_Day"] = df["Time_of_Day"].map(IDX2TIME)

    print("Location:", loc, "# of instances:", len(df))
    print("Min:", df["power"].min(), "Max:", df["power"].max(), "Avg:", df["power"].mean())
    # print("Weekday Min:", weekday_df["power"].min(), "Weekday Max:", weekday_df["power"].max(), "Weekday Avg:", weekday_df["power"].mean())
    # print("Weekend Min:", weekend_df["power"].min(), "Weekend Max:", weekend_df["power"].max(), "Weekend Avg:", weekend_df["power"].mean())
    # print("Morn Min:", morn_df["power"].min(), "Morn Max:", morn_df["power"].max(), "Morn Avg:", morn_df["power"].mean())
    # # print("Noon Min:", noon_df["power"].min(), "Noon Max:", noon_df["power"].max(), "Noon Avg:", noon_df["power"].mean())
    # # print("Eve Min:", eve_df["power"].min(), "Eve Max:", eve_df["power"].max(), "Eve Avg:", eve_df["power"].mean())
    # print("Night Min:", night_df["power"].min(), "Night Max:", night_df["power"].max(), "Night Avg:", night_df["power"].mean())
    # print("% of Morning frequency to Night:", get_avg(morn_df["power"].mean(), night_df["power"].mean()))
    json_ip[f"power_{loc}"] = df["power"].to_list()
    # json_ip[f"hour_{loc}"] = df["Hour"].to_list()
    print(json_ip[f"power_{loc}"])
    # json_ip[f"date_{loc}"] = df["Date"].to_list()
    # json_ip[f"day_{loc}"] = df["Day"].to_list()
    print("\n")


    # Compare which location is best for running data

['2023-11-26', '2023-11-26', '2023-11-27', '2023-11-27', '2023-11-27', '2023-11-27', '2023-11-28', '2023-11-28', '2023-11-28', '2023-11-29', '2023-11-29', '2023-11-29', '2023-11-29', '2023-11-30', '2023-11-30', '2023-11-30', '2023-12-01', '2023-12-01', '2023-12-01', '2023-12-01', '2023-12-02', '2023-12-02', '2023-12-02', '2023-12-12', '2023-12-12', '2023-12-13', '2023-12-13', '2023-12-13', '2023-12-13', '2023-12-14', '2023-12-14', '2023-12-15', '2023-12-15', '2023-12-15', '2023-12-15', '2023-12-16', '2023-12-16', '2023-12-16', '2023-12-17', '2023-12-17', '2023-12-17', '2023-12-18', '2023-12-18', '2023-12-18', '2023-12-18', '2023-12-19', '2023-12-19', '2023-12-19', '2023-12-20', '2023-12-20', '2023-12-20', '2023-12-20', '2023-12-21', '2023-12-21', '2023-12-21', '2023-12-22', '2023-12-22', '2023-12-22', '2023-12-23', '2023-12-23', '2023-12-23', '2023-12-23', '2023-12-24', '2023-12-24', '2023-12-24', '2023-12-25', '2023-12-25', '2023-12-25', '2023-12-25', '2023-12-26', '2023-12-26', '2023

## Location-wise weather analysis

In [33]:
df['wdir'].unique()

array([260., 140., 170., 310., 180., 160., 150.,  30., 110.,   0., 360.,
       270., 130., 100., 300., 290., 320.,  10., 190.,  80., 330., 340.,
       350., 250., 240.,  20., 200.,  50., 280., 120., 230., 326., 122.,
       306., 210.,  70., 175., 162., 164., 166.,  60., 303., 248., 307.,
       134., 172., 149., 169., 236., 328., 301., 309.,  40.,  90., 319.,
       220., 308., 331., 153., 272.])

In [38]:
folders = ["data_0", "data_1", "data_2", "data_3", "data_4", "data_5", "data_6", "data_7"] # "data_0", "data_1", "data_2", "data_3", "data_4", "data_5", "data_6", "data_7"
json_ip = {}
for folder in folders:
    df = pd.read_csv(f"compressed_data/compressed_{folder}.csv")

    feat = "pres" # temp, rhum, prcp, pres, dwpt, wdir, wspd, coco

    df1 = df.dropna()
    df1 = df1.sort_values(by=feat)
    print(folder)
    if feat in ['temp', 'pres', 'dwpt']:
        df1[feat] = df[feat].round(0).astype(int)
    df1 = df1.groupby(feat, as_index=False)['power'].mean()

    r, p = pearsonr(df1["power"], df1[feat])
    print("Correlation coefficient (r):", r, "P-value:", p)

    json_ip[f"power_{folder[-1]}"] = df1["power"].to_list()
    json_ip[f"temperature"] = df1[feat].to_list()

data_0
Correlation coefficient (r): 0.8934133653161487 P-value: 1.3449226707187412e-17
data_1
Correlation coefficient (r): 0.7558578272601715 P-value: 5.275309302645175e-10
data_2
Correlation coefficient (r): -0.12495218639836934 P-value: 0.39744219260095
data_3
Correlation coefficient (r): 0.8122334750426865 P-value: 2.4328236261684498e-12
data_4
Correlation coefficient (r): -0.1833140862091328 P-value: 0.24522011032372837
data_5
Correlation coefficient (r): 0.8608143382090727 P-value: 1.706634306885268e-14
data_6
Correlation coefficient (r): 0.7020110187990705 P-value: 2.7022113886582227e-08
data_7
Correlation coefficient (r): 0.6690553619912776 P-value: 2.010893185690373e-07


In [39]:
np.sqrt(mean_squared_error([0.8934, 0.7559, -0.1250, 0.8122, -0.1833, 0.8608, 0.7020, 0.6691], [0.8242, 0.7651, 0.0889, 0.7559, -0.0989, 0.8456, 0.7651, 0.7651]))

np.float64(0.09640292397017841)

In [36]:
pred_dict = {
  "location_0": {
      'actual_p': 7.984042193306028e-12,
      'pred_p': 5.41e-16,
      'actual_r': -0.8011739915387832,
      'pred_r': -0.8783
  },
  "location_1": {
      'actual_p': 1.0414716818326217e-18,
      'pred_p': 8.04e-17,
      'actual_r': -0.9051707742883124,
      'pred_r': -0.8907
  },
  "location_2": {
      'actual_p': 0.28156722625263275,
      'pred_p': 2.05e-01,
      'actual_r': -0.15692488508556654,
      'pred_r': -0.1847
  },
  "location_3": {
      'actual_p': 2.139842508347913e-08,
      'pred_p': 8.01e-06,
      'actual_r': -0.7002518074578323,
      'pred_r': -0.5919
  },
  "location_4": {
      'actual_p': 2.6425619996861594e-07,
      'pred_p': 2.02e-17,
      'actual_r': -0.6696628415564131,
      'pred_r': -0.9138
  },
  "location_5": {
      'actual_p': 1.6855442796473395e-19,
      'pred_p': 2.86e-17,
      'actual_r': -0.916292385504006,
      'pred_r': -0.9118
  },
  "location_6": {
      'actual_p': 2.3337626161065696e-22,
      'pred_p': 2.05e-20,
      'actual_r': -0.9351254820287205,
      'pred_r': -0.9231
  },
  "location_7": {
      'actual_p': 1.2651798015514928e-23,
      'pred_p': 8.93e-21,
      'actual_r': -0.9430610266309578,
      'pred_r': -0.9269
  }
}


from sklearn.metrics import mean_absolute_error, root_mean_squared_error
import numpy as np
from scipy.stats import pearsonr

actual_r = [v['actual_r'] for v in pred_dict.values()]
pred_r = [v['pred_r'] for v in pred_dict.values()]

mae = mean_absolute_error(actual_r, pred_r)
rmse = root_mean_squared_error(actual_r, pred_r)
r_corr, _ = pearsonr(actual_r, pred_r)

print(f"MAE (r): {mae:.4f}")
print(f"RMSE (r): {rmse:.4f}")
print(f"Pearson Correlation (r vs pred_r): {r_corr:.4f}")

MAE (r): 0.0631
RMSE (r): 0.0992
Pearson Correlation (r vs pred_r): 0.9241


In [ ]:
pred_dict ={
  "location_0": {
    "measurement_count": 48,
    "attempt_1": -98.80686761474609,
    "attempt_2": -98.84975173152736,
    "attempt_3": -98.84975173152736,
    "actual_power": -98.88355541492905
  },
  "location_1": {
    "measurement_count": 48,
    "attempt_1": -93.91979029133225,
    "attempt_2": -93.97028935338146,
    "attempt_3": -93.97028935338147,
    "actual_power": -93.88549704677419
  },
  "location_2": {
    "measurement_count": 49,
    "attempt_1": -97.61234133666538,
    "attempt_2": -97.6029345053694,
    "attempt_3": -97.60293450536939,
    "actual_power": -97.55560029501878
  },
  "location_3": {
    "measurement_count": 49,
    "attempt_1": -99.45659929282449,
    "attempt_2": -99.4519217465413,
    "attempt_3": -99.45192174654131,
    "actual_power": -99.3956621382895
  },
  "location_4": {
    "measurement_count": 47,
    "attempt_1": -95.32042797129918,
    "attempt_2": -95.31106204431053,
    "attempt_3": -95.31106204431054,
    "actual_power": -95.08064362233789
  },
  "location_5": {
    "measurement_count": 47,
    "attempt_1": -102.08753985183121,
    "attempt_2": -102.0897057825639,
    "attempt_3": -102.08970578256389,
    "actual_power": -101.81935770354782
  },
  "location_6": {
    "measurement_count": 48,
    "attempt_1": -93.53493602275848,
    "attempt_2": -93.53485444633268,
    "attempt_3": -93.53485444633268,
    "actual_power": -93.39825489566533
  },
  "location_7": {
    "measurement_count": 48,
    "attempt_1": -97.71304348052486,
    "attempt_2": -97.70947494082723,
    "attempt_3": -97.70947494082724,
    "actual_power": -97.62343619337052
  }
}

def calculate_errors(pred_dict, attempt_key, actual_key="actual_power"):
    attempt_vals = []
    actual_vals = []

    for location, metrics in pred_dict.items():
        if attempt_key in metrics and actual_key in metrics:
            attempt_vals.append(metrics[attempt_key])
            actual_vals.append(metrics[actual_key])

    attempt_vals = np.array(attempt_vals)
    actual_vals = np.array(actual_vals)

    mae = np.mean(np.abs(attempt_vals - actual_vals))
    rmse = np.sqrt(np.mean((attempt_vals - actual_vals) ** 2))
    mape = np.mean(np.abs((attempt_vals - actual_vals) / actual_vals)) * 100

    return {
        "MAE": mae,
        "RMSE": rmse,
        "MAPE (%)": mape
    }

calculate_errors(pred_dict, "attempt_3")


In [3]:
for idx, file in tqdm(enumerate(os.listdir('./compressed_data')[:8])):
    print(file)
    df = pd.read_csv('./compressed_data/'+file)
    df = df[['power', 'Date']]
    df['Date'] = pd.to_datetime(df['Date'])
    data_daily = df.groupby(df['Date'].dt.date).agg({'power': 'mean'}).reset_index()
    date_list = [d.isoformat() for d in data_daily['Date'].to_list()]
    print(date_list)
    print(data_daily['power'].to_list())
    # data.set_index('Date', inplace=True)
    # data = data.resample('1H').mean()

0it [00:00, ?it/s]

compressed_data_0.csv


0it [00:07, ?it/s]


KeyboardInterrupt: 

In [40]:
df = pd.read_csv('compressed_data/compressed_data_0.csv')
df['Date'] = pd.to_datetime(df['Date'])
data_daily = df.groupby(df['Date'].dt.date).agg({'power': 'mean'}).reset_index()
data_daily['Date'] = pd.to_datetime(data_daily['Date'])
data_daily['day_of_week'] = data_daily['Date'].dt.day_name()
daily_avg = data_daily.groupby('day_of_week')['power'].mean().reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
print("Daily Average:", daily_avg, '\n')

# data_daily['week_number'] = data_daily['Date'].dt.isocalendar().week
# weekly_avg = data_daily.groupby('week_number')['power'].mean()
# print("Weekly Average (Week 1-52):", weekly_avg, '\n')

data_daily['month'] = data_daily['Date'].dt.month_name()
monthly_avg = data_daily.groupby('month')['power'].mean().reindex(['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'])
print("Monthly Average (Jan-Dec):", monthly_avg, '\n')

data_daily['quarter'] = data_daily['Date'].dt.quarter
quarterly_avg = data_daily.groupby('quarter')['power'].mean()
print("Quarterly Average (Q1-Q4):", quarterly_avg)

Daily Average: day_of_week
Monday      -98.706741
Tuesday     -98.591083
Wednesday   -98.692454
Thursday    -98.725372
Friday      -98.692429
Saturday    -98.977410
Sunday      -98.984293
Name: power, dtype: float64 

Monthly Average (Jan-Dec): month
January      -97.610555
February     -98.824204
March        -99.435910
April        -99.873126
May         -100.062003
June         -99.636535
July         -99.415845
August       -98.827673
September    -99.285192
October      -98.717710
November     -97.714210
December     -97.680737
Name: power, dtype: float64 

Quarterly Average (Q1-Q4): quarter
1   -98.428598
2   -99.859472
3   -99.182772
4   -97.967958
Name: power, dtype: float64


In [47]:
# for actual_list in ['daily_avg', 'weekly_avg', 'monthly_avg', 'quarterly_avg']:
np.sqrt(mean_squared_error(daily_avg, [-98.57, -98.56, -98.58, -98.59, -98.63, -98.73, -98.57]))

np.float64(0.20261986050744438)

In [23]:
def create_message(json_ip):
    return [
        {"role": "user", 
        # "content": f"""

        # # Power Data Analysis Task

        # You are a Data Analyst tasked with analyzing time-series power data collected from four different locations at 2.4 GHz frequency over several months. 

        # ## Data Description:
        # - Format: JSON with "power_n" arrays containing power readings for location n
        # - "hour_n" arrays contain corresponding hour from timestamp for each reading
        # - All power readings are in dBm (decibel-milliwatts)

        # ## Data:
        # {json_ip}

        # ## Analysis Tasks:

        # 1. DATA VALIDATION AND COUNTS
        # - First, explicitly count and report the exact number of values in each power_n array
        # - Verify that hour_n arrays match in length with their corresponding power_n arrays
        # - Report any discrepancies or anomalies in the data structure

        # 2. STATISTICS BY LOCATION
        # - Calculate precise min, max, average, and standard deviation of power values for each location
        # - For min and max values, identify and show the exact indices and hour timestamps where they occur
        # - Verify calculations by showing the 5 highest and 5 lowest values for each location

        # 3. TIME-OF-DAY PATTERNS
        # - Define morning (7 to 18 inclusive), night (19 to 6 inclusive)
        # - Calculate average power for each time period at each location
        # - Show the sample count for each time period to verify correct categorization
        # - Calculate the percentage difference between morning and night averages (using formula: ((morning_avg - night_avg)/night_avg)*100)
        # - Calculate the ratio of morning samples to night samples
        # - Identify which time period shows the highest and lowest power readings at each location
        # - Note any locations that deviate from overall patterns

        # ## Output Requirements:
        # - Include EXACT numerical values in your calculations (no rounding, display to 6 decimal places)
        # - Show your reasoning step-by-step, including validation checks
        # - For critical values like min/max, include the specific data point index
        # - Present final results in a clear, tabular format where appropriate
        # - Ensure all percentages are calculated correctly as ((new-old)/old)*100

        # """}
        "content": f"""
        I have time-series "power" data collected at a frequencies 2412Hz, 2437Hz, 2462Hz, and 2484Hz from a single location. For each frequency, I've compiled a chronologically ordered list of power measurements taken at this location on different days and times.

        Here's my data in the form of JSON. "power_n" represents the power values at nth frequency and date_n represents the date and time at which the corresponding power is captured:
        {json_ip}

        ## Analysis Tasks:

        1. DATA VALIDATION AND COUNTS
        - First, explicitly count and report the exact number of values in each power_n array
        - Verify that hour_n arrays match in length with their corresponding power_n arrays
        - Report any discrepancies or anomalies in the data structure

        2. STATISTICS BY LOCATION
        - Calculate precise min, max, average, and standard deviation of power values for each location
        - For min and max values, identify and show the exact indices and hour timestamps where they occur
        - Verify calculations by showing the 5 highest and 5 lowest values for each location

        3. TEMPORAL PATTERNS
        Analyse the Power values at each frequency and tell me if you see any daily, weekly, monthly or quarterly patterns among the frequencies. Please specify the time ranges where you see some specific pattern and what pattern you observe.

        """}

        
        # - "day_n" arrays represent the days of the week for each reading
        # - All timestamps are in ISO format (YYYY-MM-DD HH:MM:SS)
        # - Data spans multiple days with multiple readings per day

        # 2. DAY TYPE PATTERNS
        # - Separate data into weekday vs weekend readings for each location
        # - Calculate average and standard deviation for each category
        # - Test if there's a statistically significant difference between weekday and weekend power levels
        # - Report percentage difference between weekday and weekend averages
    ]

def claude_call(json_ip):
    load_dotenv()
    CLAUDE_API_KEY = os.getenv('CLAUDE_API_KEY')
    client = anthropic.Anthropic(api_key=CLAUDE_API_KEY)
    prompt = create_message(json_ip)
    # print(prompt)
    response = client.messages.create(model="claude-3-7-sonnet-20250219", temperature=0, messages=create_message(json_ip), max_tokens=4096)
    return response.content[0].text

op = claude_call(json_ip)

sections = op.split("\n\n")
for i, section in enumerate(sections, 1):
    print(f"\n{section}\n")


# WiFi Signal Power Analysis Report


## 1. Data Validation and Counts


### Count of Values in Each Power Array
- **power_2411.978**: 1,000 measurements
- **power_2436.981**: 1,000 measurements
- **power_2461.984**: 1,000 measurements
- **power_2483.987**: 700 measurements


### Validation of Date-Power Pairs
- **2411.978 MHz**: 1,000 power values with 1,000 matching timestamps ✓
- **2436.981 MHz**: 1,000 power values with 1,000 matching timestamps ✓
- **2461.984 MHz**: 1,000 power values with 1,000 matching timestamps ✓
- **2483.987 MHz**: 700 power values with 700 matching timestamps ✓


### Data Structure Anomalies
- The 2483.987 MHz frequency has 30% fewer measurements (700) compared to the other frequencies (1,000 each)
- No missing values or mismatched pairs were detected within the provided data
- All power values appear to be in the expected range for WiFi signal strength measurements (approximately -90 to -104 dBm)


## 2. Statistics by Frequency


### 2411.978 MHz (Channel 